# Lesson I

## Real-world Examples

You've learned a lot about how decorators work. This lesson will walk you through some real-world decorators so that you can start to recognize common decorator patterns.

### Time a function

The ``timer()`` decorator runs the decorated function and then prints how long it took for the function to run. I usually wind up adding some version of this to all of my projects because it is a pretty easy way to figure out where your computational bottlenecks are. 

```python
    import time

    def timer(func):
        """A decorator that prints how long a function took to run.

        Args:
            func (callable): The function being decorated.

        Returns:
            callable: The decorated function.
        """    
```

All decorators have fairly similar-looking docstrings because they all take and return a single function. For brevity, I will only include the description of the function in the docstrings of the examples that follow.



In [6]:
import time

def timer(func):
    """A decorator that prints how long a function took to run."""
    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # When wrapper() is called, get the current time
        t_start = time.time()
        # Call the decorated function and store the result.
        result = func(*args, **kwargs)
        # Get the total time it took to run, and print it.
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        return result
    return wrapper
    

Like most decorators, we'll start off by defining a ``wrapper()`` function. This is the function that the decorator will return. ``wrapper()`` takes any number of positional and keyword arguments so that it can be used to decorate any function. 

The first thing the new function will do is record the time that it was called with the ``time()`` function. Then ``wrapper()`` gets the result of calling the decorated function. We don't return that value yet though. 

After calling the decorated function, ``wrapper()`` checks the time again, and prints a message about how long it took to run the decorated function. Once we've done that, we need to return the value that the decorated function calculated.

In [7]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)
    
print(sleep_n_seconds(5))

print(sleep_n_seconds(10))    

sleep_n_seconds took 5.011141538619995s
None
sleep_n_seconds took 10.015560150146484s
None


So if we decorate this simple ``sleep_n_seconds()`` function, you can see that sleeping for 5 seconds takes about 5 seconds, and sleeping for 10 seconds takes about 10 seconds. This is a trivial use of the decorator to show it working, but it can be very useful for finding the slow parts of your code.

## Memoizing

Memoizing is the process of storing the results of a function so that the next time the function is called with the same arguments; you can just look up the answer.

In [8]:
def memoize(func):
    """Store the result of the decorated function for fast lookup
    """
    # Store results in a dict that maps arguments to results
    cache = {}
    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # If these arguments haven't been seen before,
        if (args, kwargs) not in cache:
            # Call func() and store the result.
            cache [(args, kwargs)] = func(*args, **kwargs)
        return cache[(args, kwargs)]
    return wrapper    

We start by setting up a dictionary that will map arguments to results. Then, as usual, we create ``wrapper()`` to be the new decorated function that this decorator returns. 

When the new function gets called, we check to see whether we've ever seen these arguments before. If we haven't, we send them to the decorated function and store the result in the ``"cache"`` dictionary. 

Now we can look up the return value quickly in a dictionary of results. The next time we call this function with those same arguments, the return value will already be in the dictionary.

In [ ]:
@memoize
def slow_function(a, b):
    print('Sleeping...')
    time.sleep(5)
    return a + b

slow_function(3, 4)

Here we are memoizing ``slow_function()``. ``slow_function()`` simply returns the sum of its arguments. In order to simulate a slow function, we have it sleep for 5 seconds before returning. 

If we call ``slow_function()`` with the arguments *3 and 4, it will sleep for 5 seconds and then return 7*. But if we call ``slow_function()`` with the arguments 3 and 4 again, it will immediately return 7. Because we've stored the answer in the cache, the decorated function doesn't even have to call the original ``slow_function()`` function.

## When to use decorators

So when is it appropriate to use a decorator? You should consider using a decorator when you want to add some common bit of code to multiple functions. 

* Add common behavior to multiple functions

```python
    @timer
    def foo():
        # Do some computation

    @timer
    def bar():
        # Do some computation

    @timer
    def baz():
        # Do some computation        
```

We could have added timing code in the body of all three of these functions, but that would violate the principle of Don't Repeat Yourself. Adding a decorator is a better choice.

# Lesson II

## Decorators and Metadata

One of the problems with decorators is that they obscure the decorated function's metadata. In this lesson, I'll show you why it's a problem and how to fix it.

Here we have a nice function, ``sleep_n_seconds()``, with a *docstring* that explains exactly what it does. If we look at the ``docstring`` attribute, we can see the text of the *docstring*.

In [ ]:
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
print(sleep_n_seconds.__doc__)    

Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    


We can also access other metadata for the function, like its name and default arguments.

In [ ]:
print(sleep_n_seconds.__name__)

print(sleep_n_seconds.__defaults__)

sleep_n_seconds
(10,)


But watch what happens when we decorate ``sleep_n_seconds()`` with the ``timer()`` decorator as we've done here. 

When we try to *print* the *docstring*, we get *nothing back*. Even stranger, when we try to look up the function's name, Python tells us that ``sleep_n_seconds()``'s name is ``"wrapper"``.

In [10]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
print(sleep_n_seconds.__doc__)

print(sleep_n_seconds.__name__)    

None
wrapper


To understand why we have to examine the ``timer()`` decorator. 

```python
    def timer(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f"{func.__name__} took {end - start} seconds")
            return result
        return wrapper
```

Remember that when we write decorators, we almost always define a nested function to return. Because the decorator overwrites the ``sleep_n_seconds()`` function, when you ask for ``sleep_n_seconds()``'s docstring or name, you are actually referencing the nested function that was returned by the decorator. In this case, the nested function was called ``wrapper()`` and it didn't have a docstring.

## functools.wraps()

Fortunately, Python provides us with an easy way to fix this. The ``wraps()`` function from the ``functools`` module is a decorator that you use when defining a decorator. 

If you use it to decorate the wrapper function that your decorator returns, it will modify ``wrapper()``'s metadata to look like the function you are decorating.

In [11]:
from functools import wraps
def timer(func):
    """A decorator that prints how long a function took to run"""
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        t_start = time.time()
        
        result = func(*args, **kwargs)
        
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        
        return result
    return wrapper



Notice that the ``wraps()`` decorator takes the function you are decorating as an argument.

If we use this updated version of the ``timer()`` decorator to decorate ``sleep_n_seconds()`` and then try to print ``sleep_n_seconds()``'s docstring, we get the result we expect.

In [14]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
    
    
    Args:
        n (int) : The number of seconds to pause for.
    """
    time.sleep(n)
    
    
print(sleep_n_seconds.__doc__)   

Pause processing for n seconds.
    
    
    Args:
        n (int) : The number of seconds to pause for.
    


Likewise, printing the name or any other metadata now gives you the metadata from the function being decorated rather than the metadata of the ``wrapper()`` function.

In [15]:
print(sleep_n_seconds.__name__)

print(sleep_n_seconds.__defaults__)

sleep_n_seconds
None


### Access to original function

As an added bonus, using ``wraps()`` when creating your decorator also gives you easy access to the original undecorated function via the ``__wrapped__`` attribute. Of course, you always had access to this function via the closure, but this is an easy way to get to it if you need it.

In [16]:
print(sleep_n_seconds.__wrapped__)

<function sleep_n_seconds at 0x00000200221A6700>
